In [11]:
from typing import *
import libcst as cst
import networkx as nx
import pandas as pd
import numpy as np

In [12]:
from alkh import cst_utils

In [13]:
# file_path = 'play.py'
file_path = 'play2.py'
# file_path = '/mnt/dev/azure_projects/ATool/core/aoi_defects_clustering/aoi_defect_clustering.py'

In [14]:
file_lines = open(file_path, 'r').readlines()
file_content = open(file_path, 'r').read()
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(file_content))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
ranges = wrapper.resolve(cst.metadata.PositionProvider)
file_number_of_lines = len(file_lines)

In [18]:
class AssignCollector(cst.CSTVisitor):
    def __init__(self, ranges):
        super().__init__()
        self._ranges = ranges
        self._assign_info: List[Tuple] = []

    def get_info(self):
        return self._assign_info

    def visit_Assign(self, node: cst.FunctionDef) -> None:
        pos_start = self._ranges[node].start
        pos_end = self._ranges[node].end
        value_collector = ValueCollector()
        node.value.visit(value_collector)
        value_dict = {'names': value_collector.names, 'ints': value_collector.ints, 'floats': value_collector.floats}
        for target in node.targets:
            target_collector = ValueCollector()
            target.visit(target_collector)
            names_list = target_collector.names
            if names_list:
                self._assign_info.append((target_collector.names, value_dict, pos_start.line, pos_end.line))


class ValueCollector(cst.CSTVisitor):
    def __init__(self):
        super().__init__()
        self.names: List[List] = []
        self.ints: List[str] = []
        self.floats: List[str] = []
        self._attribute_level = 0

    def visit_Name(self, node: cst.FunctionDef) -> None:
        if self._attribute_level == 0:
            self.names.append([node.value])

    def visit_Integer(self, node: cst.FunctionDef) -> None:
        self.ints.append(node.value)

    def visit_Float(self, node: cst.FunctionDef) -> None:
        self.floats.append(node.value)

    def visit_Attribute(self, node: cst.FunctionDef) -> None:
        self._attribute_level += 1
        
    def visit_Subscript(self, node: cst.FunctionDef) -> None:
        self._attribute_level += 1

    def leave_Attribute(self, node: cst.FunctionDef) -> None:
        if isinstance(node.value, cst.Name):
            self.names.append([node.value.value, node.attr.value])
        else:
            self.names[len(self.names) - 1].append(node.attr.value)
        self._attribute_level -= 1
        
    def leave_Subscript(self, node: cst.FunctionDef) -> None:
        if isinstance(node.value, cst.Name):
            self.names.append([node.value.value])
        self._attribute_level -= 1

In [19]:
assign_collector = AssignCollector(ranges)
wrapper.visit(assign_collector)
assign_collector.get_info()

[([['max_kpi']],
  {'names': [['scan_summary_df', 'max1', 'max2', 'max3'], ['klj', 'sm']],
   'ints': [],
   'floats': []},
  1,
  2)]

In [20]:
wrapper.module.body[0].body[0]

Assign(
    targets=[
        AssignTarget(
            target=Name(
                value='max_kpi',
                lpar=[],
                rpar=[],
            ),
            whitespace_before_equal=SimpleWhitespace(
                value=' ',
            ),
            whitespace_after_equal=SimpleWhitespace(
                value=' \\\n    ',
            ),
        ),
    ],
    value=Call(
        func=Attribute(
            value=Subscript(
                value=Attribute(
                    value=Attribute(
                        value=Subscript(
                            value=Name(
                                value='scan_summary_df',
                                lpar=[],
                                rpar=[],
                            ),
                            slice=[
                                SubscriptElement(
                                    slice=Index(
                                        value=SimpleString(
                               